In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import folium

In [ ]:
# {$and:[{},{}]}
# {$and:[{$or:[{},{},{},{}]},{}]}

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies_cleaned')

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [5]:
coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies_cleaned')

In [6]:
companies = list(coll.find())

In [7]:
len(companies) # 9025

9025

In [26]:
categories = []
for i in range(len(companies)):
    categories.append(companies[i]['category_code'])
print(set(categories))

{'messaging', 'music', 'fashion', 'hardware', 'design', 'travel', 'nanotech', 'education', 'social', 'enterprise', 'manufacturing', 'health', 'mobile', 'other', 'games_video', 'medical', 'finance', 'cleantech', 'search', 'security', 'transportation', 'public_relations', 'software', 'sports', None, 'photo_video', 'nonprofit', 'consulting', 'automotive', 'local', 'analytics', 'legal', 'web', 'semiconductor', 'advertising', 'hospitality', 'biotech', 'real_estate', 'ecommerce', 'network_hosting', 'news'}


In [ ]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
# 30% of the company have at least 1 child.
# Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
# Account managers need to travel a lot
# All people in the company have between 25 and 40 years, give them some place to go to party.
# The CEO is Vegan

In [19]:
active_companies = list(coll.find({'deadpooled_year': np.nan}))

In [20]:
len(active_companies) # 8406

8406

In [21]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.

# {$and:[{deadpooled_year: NaN},{founded_year:{$lte:2009}}]}

old_companies = list(coll.find({'$and':[{'deadpooled_year': np.nan},{'founded_year':{'$lte':2009}}]}))


In [22]:
len(old_companies) # 6840

6840

In [29]:
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.

# {$and:[{$or:[{category_code:'hardware'},{category_code:'nanotech'},{category_code:'mobile'},{category_code:'games_video'},{category_code:'cleantech'},{category_code:'software'},{category_code:'analytics'},{category_code:'web'},{category_code:'biotech'}]},{deadpooled_year: NaN},{founded_year:{$gte:2010}},{total_money_raised_USD:{$gte:1000000}}]}

successful_tech_startups_2010 = list(coll.find(
    {'$and':[{'$or':[{'category_code':'hardware'},{'category_code':'nanotech'},{'category_code':'mobile'},
                     {'category_code':'games_video'},{'category_code':'cleantech'},{'category_code':'software'},
                     {'category_code':'analytics'},{'category_code':'web'},{'category_code':'biotech'}]},
             {'deadpooled_year': np.nan},{'founded_year':{'$gte':2010}},{'total_money_raised_USD':{'$gte':1000000}}]}))

successful_tech_startups_2000 = list(coll.find(
    {'$and':[{'$or':[{'category_code':'hardware'},{'category_code':'nanotech'},{'category_code':'mobile'},
                     {'category_code':'games_video'},{'category_code':'cleantech'},{'category_code':'software'},
                     {'category_code':'analytics'},{'category_code':'web'},{'category_code':'biotech'}]},
             {'deadpooled_year': np.nan},{'founded_year':{'$gte':2000}},{'total_money_raised_USD':{'$gte':1000000}}]}))


In [30]:
print(len(successful_tech_startups_2010))
print(len(successful_tech_startups_2000))

11
952


In [34]:
# Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
design_companies = list(coll.find({'category_code':'design'}))

In [35]:
print(len(design_companies))

2


In [63]:
manhattan_companies = coll.find(
    {"office_1_location":{"$near":{"$geometry":{"type":"Point","coordinates":[-73.9712,40.7831]},
                                   "$maxDistance":3000}}})

# "$maxDistance":10000,"$minDistance":5000

In [64]:
manhattan_companies = list(manhattan_companies)

In [65]:
len(manhattan_companies)

56

In [ ]:
map_city = folium.Map(location=[40.7221,-73.9712], zoom_start=12)
for company in manhattan_companies:
    folium.Marker(company['office_1_location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
map_city

In [ ]:
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266
# import geopy.distance
# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)
# print geopy.distance.vincenty(coords_1, coords_2).km